Домашнее задание по теме “Синтаксический анализ”.

Задание 1: Составление словарей для классификации по тональности.

При классификации текстов или предложений по тональности необходимо использовать оценочные словари для предметной области, то есть, такие словари, в которых содержатся отрицательные и позитивные слова для какой-то предметной области. Идея подобных словарей основана на следующих наблюдениях: во-первых, для разных товаров используются разные оценочные слова (например бывает “захватывающая книга”, но не бывает “захватывающих лыж”), во-вторых, в контексте разных товаров одни и те же слова могут иметь разную окраску (слово “тормоз” в отзыве на велосипед имеет нейтральную окраску, в отзыве на компьютер – резко негативную, “пыль” в контексте пылесосов – нейтральную, в контексте кофемолок – положительную (“мелкий помол в пыль”)). Еще один пример: "теплое пиво" – это плохо, а "теплый свитер" – это хорошо.

Составление таких словарей вручную – трудоемкий процесс, но, к счастью, его не сложно автоматизировать, если собрать достаточно большие корпуса отзывов. В этом домашнем задании вам предстоит попробовать реализовать один их подходов к составлению оценочных словарей, основанный на статье Inducing Domain-Specific Sentiment Lexicons from Unlabeled Corpora (https://nlp.stanford.edu/pubs/hamilton2016inducing.pdf).

Данные для задания – уже знакомые вам отзывы на банки, собранные с нескольких сайтов Рунета. Отзывы могут быть как положительными (оценка 5), так и отрицательными (оценка 1).

1.Разбейте всю коллекцию отзывов на предложения. Лемматизируйте все слова.

2.Обучите по коллекции предложений word2vec

3.Приведите несколько удачных и неудачных примеров решения стандартных текстов для word2vec:

-тест на определение ближайших слов

-тест на аналогии (мужчина – король : женщина – королева)

-тест на определение лишнего слова.

4.Постройте несколько визуализаций:

-TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов.

Задайте координаты для нового пространства следующим образом: одна ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдите координаты названий банков в этих координатах. 

-Более формально: берем вектор слова "хорошо", вычитаем из него вектор слова "плохо", получаем новый вектор, который описывает разницу между хорошими и плохими словами. Берем вектор слова "сбербанк" и умножаем его на этот новый вектор – получаем координату по первой оси. Аналогично – для второй оси. Две координаты уже можно нарисовать на плоскости.

Задание 2: Распространение метки
Определите 5-8 позитивных слов (например, “быстрый”, “удобный”) и 5-8 негативных слов (например,“очередь”, “медленно”). Эти слова будут основной будущего оценочного словаря. Пусть позитивному классу соответствует метка 1, негативному – -1. 

Пометьте выбранные слова в лексическом графе соответствующими метками. 

Запустите любой известный вам метод распространения метки (Label Propogation) в лексическом графе. На выходе метода распространения ошибки должны быть новые слова, помеченные метками 1 и -1 – это и есть искомые оценочные слова.

Алгоритмы распространения метки устроены примерно так: пусть мы находимся в вершине, помеченной +1. С какой-то вероятностью мы переносим эту метку на соседние узлы. С меньшей вероятностью переносим ее на вершины на расстоянии два. В конце распространения метки, часть вершин оказывается помечена меткой +1, часть – -1, большая часть остается без метки.

Рекомендуемые алгоритмы распространения метки:

- graphlab.label_propagation (graphlab доступен бесплатно по образовательной лицензии)
- sklearn.semi_supervised.LabelPropagation
- sklearn.semi_supervised.LabelSpreading

Пример построения графа: см. ноутбук, размещенный в текущей папке репозитория

In [1]:
# Скачаем json-файл с отзывами
# https://drive.google.com/file/d/1OelGGXPXBinXvZnDb1Bmmxe4rtdBmqdk/view?usp=sharing
!gdown --id 1OelGGXPXBinXvZnDb1Bmmxe4rtdBmqdk

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1OelGGXPXBinXvZnDb1Bmmxe4rtdBmqdk
To: /content/banki_responses.json.bz2
100% 108M/108M [00:00<00:00, 302MB/s] 


In [2]:
import json

import bz2
import regex
from tqdm import tqdm
tqdm.pandas()
from scipy import sparse

In [3]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [03:09, 1059.57it/s]


In [5]:
# Посмотрим на пример отзыва:
responses[99]

{'author': 'ronnichka',
 'bank_license': 'лицензия № 880',
 'bank_name': 'Югра',
 'city': 'г. Саратов',
 'datetime': '2015-06-03 20:56:57',
 'num_comments': 0,
 'rating_grade': 3,
 'rating_not_checked': False,
 'text': 'Здравствуйте! Хотела написать, что мне месяц не выдают карту ко вкладу, ссылаясь на "нам же их из Самары везут" (на секундочку 5 часов езды от нашего города). Но! Прочитала, что людям 3,5 месяцев не выдают карту, и поняла, что у меня все хорошо, пока что. И подарок мне дали, и кулер в отделении есть. Так что я, конечно, готова ждать. Правда хотелось бы не очень долго.',
 'title': 'Карта ко вкладу'}

In [6]:
# загрузим json в датафрейм
df = pd.json_normalize(responses)[['author', 'bank_name', 'city', 'datetime', 'rating_grade', 'text', 'title', 'bank_license', 'num_comments', 'rating_not_checked']]
df.head()

,author,bank_name,city,datetime,rating_grade,text,title,bank_license,num_comments,rating_not_checked
0,uhnov1,Бинбанк,г. Москва,2015-06-08 12:50:54,NaN,Добрый день! Я не являюсь клиентом банка и пор...,Жалоба,лицензия № 2562,0,False
1,Foryou,Сбербанк России,г. Новосибирск,2015-06-08 11:09:57,NaN,Доброго дня! Являюсь держателем зарплатной кар...,Не могу пользоваться услугой Сбербанк он-лайн,лицензия № 1481,0,False
2,Vladimir84,Бинбанк,г. Москва,2015-06-05 20:14:28,NaN,Здравствуйте! Дублирую свое заявление от 03.0...,Двойное списание за один товар.,лицензия № 2562,1,False
3,643609,Сбербанк России,г. Ставрополь,2015-06-05 13:51:01,NaN,Добрый день!! Я открыл расчетный счет в СберБа...,Меняют проценты комиссии не предупредив и не ...,лицензия № 1481,2,False
4,anfisa-2003,ОТП Банк,г. Челябинск,2015-06-05 10:58:12,NaN,"04.03.2015 г. взяла кредит в вашем банке, заяв...",Верните денежные средства за страховку,лицензия № 2766,1,False


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153499 entries, 0 to 153498
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   author              153479 non-null  object 
 1   bank_name           153499 non-null  object 
 2   city                138325 non-null  object 
 3   datetime            153499 non-null  object 
 4   rating_grade        88658 non-null   float64
 5   text                153499 non-null  object 
 6   title               153499 non-null  object 
 7   bank_license        153498 non-null  object 
 8   num_comments        153499 non-null  int64  
 9   rating_not_checked  153499 non-null  bool   
dtypes: bool(1), float64(1), int64(1), object(7)
memory usage: 10.7+ MB


In [8]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
# загрузим знаки пунктуации
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
import re
from nltk.corpus import stopwords

In [11]:
! pip install pymorphy2
import pymorphy2

m = pymorphy2.MorphAnalyzer()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
mystopwords = stopwords.words('russian') + [
    'это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д',
    'который','прошлый','сей', 'свой', 'наш', 'мочь', 'такой'
] + list(punctuation)  #добавим знаки пунктуации к списку стоп-слов
ru_words = re.compile("[А-Яа-я]+")


def words_only(txt):
    return " ".join(ru_words.findall(txt))


def lemmatize(txt):
    try:
        return  " ".join([m.parse(w)[0].normal_form for w in txt.lower().split()])
    except:
        return " "


def remove_stopwords(txt, mystopwords = mystopwords):
    try:
        return " ".join([token for token in txt.split() if not token in mystopwords])
    except:
        return ""

# "тяжелый" препроцессинг (с лемматизацией)    
def preprocess(txt):
    return remove_stopwords(lemmatize(words_only(txt.lower())))

# "легкий" препроцессинг (без лемматизации)
def lite_preprocess(txt): 
    return remove_stopwords(words_only(txt.lower()))

In [14]:
df['rating_grade'].value_counts()

1.0    47387
5.0    14713
2.0    13509
3.0     9261
4.0     3788
Name: rating_grade, dtype: int64

Мы видим, что оценки проставлены лишь примерно в половине отзывов (88658 из 153499). 
Для обучения модели на тональность отзыва сделаем подвыборку из "сильноположительных" отзывов (с оценкой 5) и "сильноотрицательных" отзывов (с оценкой 1). Подвыборку пришлось сделать, т.к. иначе на всем массиве данных лемматизация потребует очень много времени (обработка выборки в 62 тыс.отзывов заняла в Колабе 1ч.18 мин.)

In [15]:
df_selected_responses = df[['text', 'rating_grade']].loc[(df['rating_grade'] == 5) | (df['rating_grade'] == 1)]
df_selected_responses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62100 entries, 19 to 153498
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   text          62100 non-null  object 
 1   rating_grade  62100 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.4+ MB


In [16]:
# лемматизируем выборку
df_selected_responses.text = df_selected_responses.text.astype('str').progress_apply(preprocess)

100%|██████████| 62100/62100 [1:22:10<00:00, 12.59it/s]


In [17]:
df_selected_responses.tail()

,text,rating_grade
153493,оформить банк кредитный карта январь г первый ...,1.0
153494,слишком большой банк учитывать специфика проду...,5.0
153496,уважаемый руководство банк овк проживать г ива...,1.0
153497,пробовать оформить товар кредит альфа банк ком...,1.0
153498,коротко нравиться кридеть получать милый дело ...,5.0


In [18]:
# скопируем датафрейм для экспериментов
df_selected_responses_copy = df_selected_responses.copy(deep=True)

In [19]:
df_selected_responses_copy.tail()

,text,rating_grade
153493,оформить банк кредитный карта январь г первый ...,1.0
153494,слишком большой банк учитывать специфика проду...,5.0
153496,уважаемый руководство банк овк проживать г ива...,1.0
153497,пробовать оформить товар кредит альфа банк ком...,1.0
153498,коротко нравиться кридеть получать милый дело ...,5.0


In [20]:
# токенизируем выборку отзывов перед обучением модели Word2Vec
df_selected_responses_copy.text = [word_tokenize(text) for text in tqdm(df_selected_responses_copy.text)]

100%|██████████| 62100/62100 [00:54<00:00, 1149.48it/s]


In [21]:
df_selected_responses_copy.tail()

,text,rating_grade
153493,"[оформить, банк, кредитный, карта, январь, г, ...",1.0
153494,"[слишком, большой, банк, учитывать, специфика,...",5.0
153496,"[уважаемый, руководство, банк, овк, проживать,...",1.0
153497,"[пробовать, оформить, товар, кредит, альфа, ба...",1.0
153498,"[коротко, нравиться, кридеть, получать, милый,...",5.0


In [22]:
import gensim

In [23]:
# загрузим модель Word2Vec
from gensim.models.word2vec import Word2Vec

In [24]:
# model = Word2Vec(df_selected_responses.text, workers=4, vector_size=300, min_count=3, window=5, epochs=10)
# (с параметрами выдавало ошибку на 'vector_size', поэтому обучил без параметров)
%time model = Word2Vec(df_selected_responses_copy.text)

CPU times: user 2min 26s, sys: 879 ms, total: 2min 27s
Wall time: 1min 33s


## -тест на определение ближайших слов

In [25]:
model.wv.most_similar('карта')

[('карточка', 0.890892744064331),
 ('кредитка', 0.6896994113922119),
 ('именной', 0.6514638066291809),
 ('неименной', 0.6111764907836914),
 ('пластик', 0.5982503890991211),
 ('дебетовый', 0.5894976854324341),
 ('счёт', 0.5512669086456299),
 ('виза', 0.5472228527069092),
 ('взамен', 0.5467500686645508),
 ('кк', 0.5440767407417297)]

In [26]:
model.wv.most_similar('работник')

[('сотрудник', 0.83194899559021),
 ('служащий', 0.715066134929657),
 ('сотрудница', 0.6802595853805542),
 ('персонал', 0.675336480140686),
 ('работница', 0.6615258455276489),
 ('руководитель', 0.5960763692855835),
 ('клерк', 0.5920476913452148),
 ('специалист', 0.5581834316253662),
 ('заведовать', 0.521355152130127),
 ('управлять', 0.49591031670570374)]

In [27]:
model.wv.most_similar('оформить')

[('оформляться', 0.8005063533782959),
 ('оформлять', 0.798126757144928),
 ('оформление', 0.7112826108932495),
 ('завести', 0.6889925003051758),
 ('переоформить', 0.6784923076629639),
 ('подать', 0.6503112316131592),
 ('заполнить', 0.6371333599090576),
 ('взять', 0.5660649538040161),
 ('заводить', 0.5615397691726685),
 ('подписать', 0.5540287494659424)]

In [ ]:
# странное слово "кридеть" (наверное, была тройная опечатка в слове "кредит") в последнем отзыве из выборки
# 153498	[коротко, нравиться, кридеть, получать, милый,...	5.0
model.wv.most_similar('кридеть')

# Это был неудачный пример - модель выдала ошибку OOV:  "word 'кридеть' not in vocabulary"

In [29]:
# для слова "кредит" (с правильным написанием) ближайшие слова нашлись
model.wv.most_similar('кредит')

[('автокредит', 0.7949475646018982),
 ('ипотека', 0.7072169780731201),
 ('рассрочка', 0.6794109344482422),
 ('потребкредить', 0.60428786277771),
 ('потребкредит', 0.5959705114364624),
 ('досрочно', 0.5696533918380737),
 ('ссуда', 0.55357426404953),
 ('заём', 0.5225697755813599),
 ('потреба', 0.5191614031791687),
 ('долг', 0.49673697352409363)]

## -тест на аналогии (мужчина – король : женщина – королева)

In [30]:
# пример:  trained_model.most_similar(positive=['woman', 'king'], negative=['man']) >>>  Ответ: 'queen'
model.wv.most_similar(positive=['сотрудница', 'работник'], negative=['сотрудник'])

[('работница', 0.7711745500564575),
 ('операционистка', 0.6714124083518982),
 ('заведовать', 0.6212351322174072),
 ('операционист', 0.6038684844970703),
 ('дама', 0.5847540497779846),
 ('девушка', 0.5818686485290527),
 ('служащий', 0.5617626309394836),
 ('начальница', 0.5507700443267822),
 ('кассир', 0.54535973072052),
 ('девочка', 0.5434870719909668)]

In [31]:
# зеркальный вариант тоже дал хороший результат
model.wv.most_similar(positive=['работница', 'сотрудник'], negative=['сотрудница'])

[('работник', 0.775772213935852),
 ('клерк', 0.6297992467880249),
 ('персонал', 0.613052248954773),
 ('служащий', 0.5882300138473511),
 ('руководитель', 0.5705187320709229),
 ('подчинённый', 0.559806227684021),
 ('кадр', 0.5549909472465515),
 ('специалист', 0.5255444049835205),
 ('диспетчер', 0.5110459327697754),
 ('хам', 0.5013352036476135)]

In [32]:
# "неудачный" пример (низкая схожесть - 0.4 и менее)
model.wv.most_similar(positive=['касса', 'банк'], negative=['карта'])

[('кассир', 0.43222737312316895),
 ('обед', 0.3702438473701477),
 ('уступка', 0.3458181619644165),
 ('туда', 0.3341549336910248),
 ('филиал', 0.330450177192688),
 ('окошко', 0.32646897435188293),
 ('обедать', 0.32177358865737915),
 ('напрямую', 0.3214300274848938),
 ('выписать', 0.3197430670261383),
 ('толпа', 0.31766241788864136)]

## -тест на определение лишнего слова.

In [33]:
# пример: trained_model.doesnt_match("breakfast cereal dinner lunch".split()) >>> Ответ: 'cereal'
model.wv.doesnt_match("сотрудник сотрудница работник работница кофеварка".split())

/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'кофеварка'

In [34]:
# ну вообще-то странно, что в этой компании именно Сбербанк оказался лишним, а не Citibank =)
model.wv.doesnt_match("сбербанк альфабанк втб citibank".split())

/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'сбербанк'

## 4.Постройте несколько визуализаций:

-TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов.

Задайте координаты для нового пространства следующим образом: одна ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдите координаты названий банков в этих координатах. 

-Более формально: берем вектор слова "хорошо", вычитаем из него вектор слова "плохо", получаем новый вектор, который описывает разницу между хорошими и плохими словами. Берем вектор слова "сбербанк" и умножаем его на этот новый вектор – получаем координату по первой оси. Аналогично – для второй оси. Две координаты уже можно нарисовать на плоскости.

In [25]:
# Не совсем понял, каким образом сделать визуализацию TSNE для топ-100 слов.
# Для визуализации частично использовал способ, описанный по ссылке: https://nuancesprog.ru/p/14951/

from sklearn.manifold import TSNE

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_selected_responses.text)  #используем  датафрейм после лемматизации, но до токенизации

In [28]:
X.shape

(62100, 65580)

In [29]:
# количество признаков достаточно большое,
# поэтому для выбора главных компонент загрузим TruncatedSVD, т.к. "обычный" метод PCA работает для плотных матриц, а наш вектор - разреженная матрица
from sklearn.decomposition import TruncatedSVD

In [32]:
# уменьшим до 10-и признаков
X_TSVD = TruncatedSVD(n_components=10).fit_transform(X)

In [33]:
# если правильно понимаю, то для двумерной визуализации надо уменьшить кол-во компонент до 2-х
X_TSNE = TSNE(n_components=2, perplexity=10, random_state=42).fit_transform(X_TSVD)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [38]:
X_TSNE.shape

(62100, 2)

In [39]:
import plotly.express as px # для визуализации данных

In [45]:
# Создание диаграммы разброса
fig = px.scatter(None, x=X_TSNE[:,0], y=X_TSNE[:,1], 
                 labels={
                     "x": "Dimension 1",
                     "y": "Dimension 2",
                 },
                 opacity=1, color=X_TSNE[:,1])

# Изменение цвета фона графика
fig.update_layout(dict(plot_bgcolor = 'white'))

# Обновление линий осей
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black')

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black')

# Установка названия рисунка
fig.update_layout(title_text="t-SNE")

# Обновление размера маркера
fig.update_traces(marker=dict(size=3))

fig.show()

Получилось что-то таинственное и красивое =)) но с интерпретацией затрудняюсь, т.к. осмысленные метки у каждой точки отсутствуют... Видно, что четко выраженные кластеры также отсутcтвуют, но наблюдаются неоднородности в классификации, выглядящие как "пробелы", "протоки".

## Задание 2: Распространение метки
Определите 5-8 позитивных слов (например, “быстрый”, “удобный”) и 5-8 негативных слов (например,“очередь”, “медленно”). Эти слова будут основной будущего оценочного словаря. Пусть позитивному классу соответствует метка 1, негативному – -1. 

Пометьте выбранные слова в лексическом графе соответствующими метками. 

Запустите любой известный вам метод распространения метки (Label Propogation) в лексическом графе. На выходе метода распространения ошибки должны быть новые слова, помеченные метками 1 и -1 – это и есть искомые оценочные слова.

Алгоритмы распространения метки устроены примерно так: пусть мы находимся в вершине, помеченной +1. С какой-то вероятностью мы переносим эту метку на соседние узлы. С меньшей вероятностью переносим ее на вершины на расстоянии два. В конце распространения метки, часть вершин оказывается помечена меткой +1, часть – -1, большая часть остается без метки.

Рекомендуемые алгоритмы распространения метки:

- graphlab.label_propagation (graphlab доступен бесплатно по образовательной лицензии)
- sklearn.semi_supervised.LabelPropagation
- sklearn.semi_supervised.LabelSpreading

In [ ]:
# построим граф, воспользовавшись библиотекой iGraph и примером из репозитория к практике
pip install igraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.1 MB 7.0 MB/s 


In [ ]:
import igraph as ig
g = ig.Graph(directed=True)
for word in model.wv.vocab.keys():
    g.add_vertex(word)

In [ ]:
for word in model.wv.vocab.keys() :
    node = g.vs.select(name = word).indices[0]
    similar_words = model.most_similar(word, topn=5)
    for sim in similar_words:
        word1 = sim[0]
        val  = sim[1]
        new_node = g.vs.select(name = word1).indices[0]
        g.add_edge(node, new_node, weight = val)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
ig.summary(g)

IGRAPH DNW- 24137 120685 -- 
+ attr: name (v), weight (e)


In [ ]:
# сначала выведем граф в текстовый файл, т.к. по причине большого объема данных вывести функцией print в Гугл Колабе нормально не получилось
from contextlib import redirect_stdout

with open('out.txt', 'w') as f:
    with redirect_stdout(f):
        print(g)

In [ ]:
# теперь считаем из файла и выведем первые 100 позиций
with open('out.txt') as f:
  l = [line.strip() for line in f]

In [ ]:
l[:100]

['IGRAPH DNW- 24137 120685 --',
 '+ attr: name (v), weight (e)',
 '+ edges (vertex names):',
 'открыть -> открывать, открытие, открываться, откроить,',
 'отрыть',
 'вклад -> счёт, депозит, доходный, капитализация,',
 'депозитный',
 'счёт -> сч, депозит, картсчёт, сберкнижка, скс',
 'плюс -> минус, жирный, радовать, преимущество, удобство',
 'зарплатный -> дебетовый, зп, маэстро, заплатать, пилотный',
 'карта -> дебетовый, карточка, кредитка, именной,',
 'неименной',
 'рубль -> руб, р, тысяча, копейка, евро',
 'сегодня -> вчера, завтра, пятница, понедельник, вторник',
 'прийти -> приехать, приходить, отправить, поехать,',
 'прислать',
 'указанный -> указать, вышеуказанный, обозначить, указываться,',
 'сообщаться',
 'отделение -> офис, филиал, осб, отд, допофис',
 'цель -> необходимость, необходимо, путём, заодно,',
 'незадолго',
 'пополнить -> открыть, положить, пополнение, пополнять,',
 'перекинуть',
 'долларовый -> выя, рублёвый, евровыя, мультивалютный, евро',
 'сч -> счёт, расча, ка